In [2]:
import pandas as pd
df=pd.read_csv("montreal.csv")
df

,datetime,Description,Humidity,Wind Direction,Temperature,Pressure,Wind Speed
0,2012-10-01 13:00:00,overcast clouds,93.0,230.0,285.830000,1001.0,4.0
1,2012-10-01 14:00:00,sky is clear,91.0,230.0,285.834650,986.0,4.0
2,2012-10-01 15:00:00,sky is clear,87.0,231.0,285.847790,945.0,4.0
3,2012-10-01 16:00:00,sky is clear,84.0,233.0,285.860929,904.0,4.0
4,2012-10-01 17:00:00,sky is clear,80.0,234.0,285.874069,863.0,3.0
...,...,...,...,...,...,...,...
45247,2017-11-29 20:00:00,broken clouds,64.0,300.0,275.130000,1021.0,10.0
45248,2017-11-29 21:00:00,scattered clouds,59.0,300.0,274.130000,1023.0,7.0
45249,2017-11-29 22:00:00,few clouds,66.0,290.0,273.480000,1024.0,7.0
45250,2017-11-29 23:00:00,few clouds,58.0,290.0,272.480000,1026.0,4.0


In [3]:
#raw data - there are 38 unique instances...
df["Description"].value_counts()

broken clouds                   8151
sky is clear                    7299
overcast clouds                 6637
few clouds                      6317
light rain                      4015
scattered clouds                3407
mist                            3196
light snow                      1796
light intensity shower rain     1067
moderate rain                    770
heavy snow                       536
fog                              456
light shower snow                331
proximity shower rain            281
snow                             175
haze                             171
heavy intensity rain             168
heavy shower snow                119
light intensity drizzle          117
thunderstorm with light rain      69
proximity thunderstorm            39
thunderstorm                      31
smoke                             18
thunderstorm with rain            16
light intensity drizzle rain      14
shower rain                       11
light shower sleet                 9
l

In [4]:
#too much detail, simplifying a bit and cleaning/standarizing word notation
df['Description'] = df['Description'].str.replace(' with ', ' ')
df['Description'] = df['Description'].str.replace(' and ', ' ')
df['Description'] = df['Description'].str.replace('proximity ', '')
df['Description'] = df['Description'].str.replace('light intensity', 'light')
df['Description'] = df['Description'].str.replace('heavy intensity', 'heavy')
df['Description'] = df['Description'].str.replace('very heavy', 'heavy')

#exceptions / categories that the same
df['Description'] = df['Description'].str.replace('light drizzle rain', 'light rain')
df['Description'] = df['Description'].str.replace('light drizzle', 'light rain')
df['Description'] = df['Description'].str.replace('drizzle', 'light rain')
df['Description'] = df['Description'].str.replace('sleet', 'snow')
df['Description'] = df['Description'].str.replace('freezing', 'snow')
df['Description'] = df['Description'].str.replace('sand', 'other')
df['Description'] = df['Description'].str.replace('dust', 'other')
df['Description'] = df['Description'].str.replace('smoke', 'other')

#standarizing intensity values
df['Description'] = df['Description'].str.replace('few', 'light ')
df['Description'] = df['Description'].str.replace('broken', 'moderate ')
df['Description'] = df['Description'].str.replace('scattered', 'moderate ')
df['Description'] = df['Description'].str.replace('overcast ', 'heavy ')

df["Description"].value_counts()

moderate  clouds           11558
sky is clear                7299
heavy clouds                6637
light  clouds               6317
light rain                  4150
mist                        3196
light snow                  1796
light shower rain           1067
moderate rain                770
heavy snow                   536
fog                          456
light shower snow            340
shower rain                  292
snow                         175
heavy rain                   173
haze                         171
heavy shower snow            119
thunderstorm                  70
thunderstorm light rain       69
other                         21
thunderstorm rain             16
light rain snow                7
thunderstorm heavy rain        6
shower snow                    5
snow rain                      4
shower light rain              1
heavy shower rain              1
Name: Description, dtype: int64

In [5]:
#a filter to check values
df[df['Description'].str.contains("drizzle")]['Description'].value_counts()

Series([], Name: Description, dtype: int64)

In [6]:
#multi-categorical dummification
tags = ['clouds','rain','mist','snow','shower','thunderstorm','fog','other']

for i in range(len(tags)):
    df[tags[i]]=0
    df.loc[df['Description'].str.contains(pat=tags[i])==True, tags[i]] = 1

In [7]:
#creating a weather intensity column
intensity_values=['sky is clear','light','moderate','heavy']    

for i in range(len(intensity_values)):
    df.loc[df['Description'].str.contains(pat =intensity_values[i])==True, 'Intensity'] = i
    
    
#fill in the blanks with moderate
df['Intensity']=df['Intensity'].fillna(2)

In [8]:
pd.set_option('display.max_columns', None)
display(df)

df.to_csv("weather_desc_clean.csv",index=False)

,datetime,Description,Humidity,Wind Direction,Temperature,Pressure,Wind Speed,clouds,rain,mist,snow,shower,thunderstorm,fog,other,Intensity
0,2012-10-01 13:00:00,heavy clouds,93.0,230.0,285.830000,1001.0,4.0,1,0,0,0,0,0,0,0,3.0
1,2012-10-01 14:00:00,sky is clear,91.0,230.0,285.834650,986.0,4.0,0,0,0,0,0,0,0,0,0.0
2,2012-10-01 15:00:00,sky is clear,87.0,231.0,285.847790,945.0,4.0,0,0,0,0,0,0,0,0,0.0
3,2012-10-01 16:00:00,sky is clear,84.0,233.0,285.860929,904.0,4.0,0,0,0,0,0,0,0,0,0.0
4,2012-10-01 17:00:00,sky is clear,80.0,234.0,285.874069,863.0,3.0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45247,2017-11-29 20:00:00,moderate clouds,64.0,300.0,275.130000,1021.0,10.0,1,0,0,0,0,0,0,0,2.0
45248,2017-11-29 21:00:00,moderate clouds,59.0,300.0,274.130000,1023.0,7.0,1,0,0,0,0,0,0,0,2.0
45249,2017-11-29 22:00:00,light clouds,66.0,290.0,273.480000,1024.0,7.0,1,0,0,0,0,0,0,0,1.0
45250,2017-11-29 23:00:00,light clouds,58.0,290.0,272.480000,1026.0,4.0,1,0,0,0,0,0,0,0,1.0


In [9]:
#here is the result in a custom function
def clean_description(df):
    #too much detail, simplifying a bit and cleaning/standarizing word notation
    df['Description'] = df['Description'].str.replace(' with ', ' ')
    df['Description'] = df['Description'].str.replace(' and ', ' ')
    df['Description'] = df['Description'].str.replace('proximity ', '')
    df['Description'] = df['Description'].str.replace('light intensity', 'light')
    df['Description'] = df['Description'].str.replace('heavy intensity', 'heavy')
    df['Description'] = df['Description'].str.replace('very heavy', 'heavy')

    #exceptions / categories that the same
    df['Description'] = df['Description'].str.replace('light drizzle rain', 'light rain')
    df['Description'] = df['Description'].str.replace('light drizzle', 'light rain')
    df['Description'] = df['Description'].str.replace('drizzle', 'light rain')
    df['Description'] = df['Description'].str.replace('sleet', 'snow')
    df['Description'] = df['Description'].str.replace('freezing', 'snow')
    df['Description'] = df['Description'].str.replace('sand', 'other')
    df['Description'] = df['Description'].str.replace('dust', 'other')
    df['Description'] = df['Description'].str.replace('smoke', 'other')

    #standarizing intensity values
    df['Description'] = df['Description'].str.replace('few', 'light ')
    df['Description'] = df['Description'].str.replace('broken', 'moderate ')
    df['Description'] = df['Description'].str.replace('scattered', 'moderate ')
    df['Description'] = df['Description'].str.replace('overcast ', 'heavy ')
    
    #multi-categorical dummification
    tags = ['clouds','rain','mist','snow','shower','thunderstorm','fog','other']

    for i in range(len(tags)):
        df[tags[i]]=0
        df.loc[df['Description'].str.contains(pat=tags[i])==True, tags[i]] = 1
        
    #creating a weather intensity column
    intensity_values=['sky is clear','light','moderate','heavy']    

    for i in range(len(intensity_values)):
        df.loc[df['Description'].str.contains(pat =intensity_values[i])==True, 'Intensity'] = i


    #fill in the blanks with moderate
    df['Intensity']=df['Intensity'].fillna(2)
    
    return df

In [10]:
clean_description(df)

,datetime,Description,Humidity,Wind Direction,Temperature,Pressure,Wind Speed,clouds,rain,mist,snow,shower,thunderstorm,fog,other,Intensity
0,2012-10-01 13:00:00,heavy clouds,93.0,230.0,285.830000,1001.0,4.0,1,0,0,0,0,0,0,0,3.0
1,2012-10-01 14:00:00,sky is clear,91.0,230.0,285.834650,986.0,4.0,0,0,0,0,0,0,0,0,0.0
2,2012-10-01 15:00:00,sky is clear,87.0,231.0,285.847790,945.0,4.0,0,0,0,0,0,0,0,0,0.0
3,2012-10-01 16:00:00,sky is clear,84.0,233.0,285.860929,904.0,4.0,0,0,0,0,0,0,0,0,0.0
4,2012-10-01 17:00:00,sky is clear,80.0,234.0,285.874069,863.0,3.0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45247,2017-11-29 20:00:00,moderate clouds,64.0,300.0,275.130000,1021.0,10.0,1,0,0,0,0,0,0,0,2.0
45248,2017-11-29 21:00:00,moderate clouds,59.0,300.0,274.130000,1023.0,7.0,1,0,0,0,0,0,0,0,2.0
45249,2017-11-29 22:00:00,light clouds,66.0,290.0,273.480000,1024.0,7.0,1,0,0,0,0,0,0,0,1.0
45250,2017-11-29 23:00:00,light clouds,58.0,290.0,272.480000,1026.0,4.0,1,0,0,0,0,0,0,0,1.0
